In [3]:
import tensorflow as tf
from keras.layers import LSTM,Dense,Embedding
from keras.models import Sequential
import pandas as pd
import numpy as np

In [ ]:
voc_size=5000


In [18]:
df=pd.read_csv('/content/train.csv')
df.head(2)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0


In [5]:
df.isna().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [6]:
df.shape

(20800, 5)

In [7]:
df.dropna(inplace=True)

In [8]:
df.shape

(18285, 5)

In [15]:
x=df.iloc[:,0:4]
y=df['label']

In [16]:
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [17]:
print(f' x shape ={x.shape}, y shape ={y.shape}')

 x shape =(18285, 4), y shape =(18285,)


In [21]:
msg=x.copy()
msg['title'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [22]:
msg.reset_index(inplace=True)

In [24]:
import re
import nltk
from nltk.corpus import stopwords

In [25]:
from nltk.corpus import brown

In [32]:
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [36]:
msg['title'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [33]:
from nltk.stem.porter import PorterStemmer

In [43]:
ps =PorterStemmer()
corpus=[]
for i in range(0,len(msg)):
  review=re.sub('[^a-zA-Z]',' ',msg['title'][i])
  review=review.lower()
  review=review.split()
  review=[ps.stem(word) for word in review if not word in stopwords.words('english') ]
  review=' '.join(review)
  corpus.append(review)

In [46]:
from keras.preprocessing.text import one_hot
one_hotencode=[one_hot(words,voc_size)for words in corpus]
one_hotencode[0:20]

[[3228, 2022, 4339, 2556, 1668, 4940, 1564, 3259, 494, 2104],
 [2914, 2359, 4617, 3830, 979, 2559, 1614],
 [3807, 2058, 4364, 2858],
 [1970, 4260, 1168, 4815, 1993, 2040],
 [3057, 979, 36, 2760, 940, 1765, 979, 2141, 2851, 325],
 [1452,
  1453,
  3736,
  741,
  1636,
  898,
  1978,
  2257,
  2321,
  2914,
  4916,
  4711,
  299,
  3082,
  1614],
 [4826, 4643, 222, 1203, 3439, 871, 2447, 957, 3020, 4402, 2286],
 [283, 2376, 476, 1959, 653, 1073, 898, 4541, 3020, 4402, 2286],
 [3983, 190, 4771, 2092, 1406, 1551, 4848, 3953, 898, 477],
 [2026, 1066, 368, 55, 4093, 3614, 3488, 2335],
 [3491, 1479, 1161, 2380, 1130, 3246, 693, 1300, 2987, 4497, 3446],
 [4815, 4222, 1668, 1551, 898, 653],
 [101, 1817, 2083, 1322, 189, 415, 611, 4815, 1070],
 [1761, 693, 3158, 457, 4471, 1680, 3410, 3020, 4402, 2286],
 [561, 3187, 4306, 6, 4667, 3020, 4402, 2286],
 [1797, 440, 2536, 4136, 4177, 4447, 1534, 2131, 1560, 4719],
 [3710, 599, 2359],
 [2587, 1962, 1378, 4749, 898, 2330, 4109, 1614],
 [2288, 756, 461

In [48]:
from keras.preprocessing.sequence import pad_sequences
embeddeddoc=pad_sequences(one_hotencode,padding='pre',maxlen=20)
embeddeddoc[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 3228,
       2022, 4339, 2556, 1668, 4940, 1564, 3259,  494, 2104], dtype=int32)

In [49]:
model=Sequential()  # intialiase mode
model.add(Embedding(voc_size,40,input_length=20))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [50]:
xfinal =np.array(embeddeddoc)  # [embeddeddoc]
yfinal=np.array(y)

In [51]:
from sklearn.model_selection import train_test_split
xtrain ,xtest ,ytrain,ytest =train_test_split(xfinal,yfinal,test_size=0.30,random_state=1)

In [53]:
print(f'shape of xtrain and xtest {xtrain.shape}, {xtest.shape}')
print(f'shape of ytrain and ytest {ytrain.shape}, {ytest.shape}')

shape of xtrain and xtest (12799, 20), (5486, 20)
shape of ytrain and ytest (12799,), (5486,)


In [54]:
model.fit(xtrain,ytrain,validation_data=(xtest,ytest),epochs=10,batch_size=64)

Epoch 1/10
200/200 [==============================] - 11s 38ms/step - loss: 0.3269 - accuracy: 0.8505 - val_loss: 0.1753 - val_accuracy: 0.9234
Epoch 2/10
200/200 [==============================] - 8s 42ms/step - loss: 0.1364 - accuracy: 0.9456 - val_loss: 0.1730 - val_accuracy: 0.9285
Epoch 3/10
200/200 [==============================] - 9s 45ms/step - loss: 0.1021 - accuracy: 0.9635 - val_loss: 0.1948 - val_accuracy: 0.9300
Epoch 4/10
200/200 [==============================] - 7s 35ms/step - loss: 0.0733 - accuracy: 0.9733 - val_loss: 0.2363 - val_accuracy: 0.9260
Epoch 5/10
200/200 [==============================] - 8s 42ms/step - loss: 0.0522 - accuracy: 0.9838 - val_loss: 0.2746 - val_accuracy: 0.9223
Epoch 6/10
200/200 [==============================] - 9s 44ms/step - loss: 0.0376 - accuracy: 0.9869 - val_loss: 0.2750 - val_accuracy: 0.9229
Epoch 7/10
200/200 [==============================] - 7s 37ms/step - loss: 0.0236 - accuracy: 0.9924 - val_loss: 0.3324 - val_accuracy: 0.918

In [65]:
ypred=np.where(model.predict(xtest)>0.5,1,0)
ypred

172/172 [==============================] - 2s 9ms/step


array([[1],
       [1],
       [0],
       ...,
       [1],
       [0],
       [0]])

In [66]:
from sklearn.metrics import confusion_matrix
confusion_matrix(ytest,ypred)

array([[2858,  268],
       [ 176, 2184]])

In [68]:
from sklearn.metrics import accuracy_score
accuracy_score(ytest,ypred)

0.9190667152752461